In [ ]:
!pip install sentencepiece==0.1.91

     |████████████████████████████████| 1.1 MB 12.6 MB/s 


In [ ]:
!pip install transformers -q
!pip install wandb -q

# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

     |████████████████████████████████| 2.9 MB 10.8 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 636 kB 31.7 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 
     |████████████████████████████████| 3.3 MB 43.5 MB/s 
     |████████████████████████████████| 1.7 MB 13.6 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 139 kB 52.2 MB/s 
     |████████████████████████████████| 180 kB 32.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [ ]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# WandB – Import the wandb library
import wandb

In [ ]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Wed Oct 13 06:48:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
# Login to wandb to log the model run and all the parameters
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### 1. Data Loader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!ls

'Colab Notebooks'				      t5_predictions.csv
 klue_info_text.xlsx				     '문서요약 텍스트_unzip'
'[데이터톤] KLUE 강의평 텍스트 데이터 khaiii.ipynb'   wandb
'[KUBIG] KLUE 강의평 전처리.ipynb'


In [ ]:
# Loading train/validation data
import json
import pandas as pd

def read_jsonl(path):
  DATA_DIR = "문서요약 텍스트_unzip"
  with open(DATA_DIR + path , 'r') as json_file:
      json_list = list(json_file)

  trains = []

  for json_str in json_list:
      line = json.loads(json_str)
      trains.append(line)
  df = pd.DataFrame(trains)
  return df

train_df = read_jsonl('/1.Training/신문기사_1.train.jsonl/train.jsonl')
val_df = read_jsonl('/2.Validation/신문기사_1.vaild.jsonl/vaild.jsonl')
train_df.head()

,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"


In [ ]:
# join으로 str로 바꿔줘야할듯
for i,info in enumerate(train_df['article_original'][1]):
  print('line {} : {}'.format(i,info))

line 0 : 서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 우수의정대상을 받았다.
line 1 : 가충순 의원과 이수의 의원은 16일 팔봉면 폰타나 리조트에서 열린 한국지역신문협회 하계 워크샵에서 지역사회 발전을 위해 활발한 의정활동을 펼친 공로를 인정받아 우수의정대상을 수상했다.
line 2 : 지난해 6월 제7회 전국동시지방선거를 통해 등원한 두 의원은 산업건설위원회에서 열정적인 의정활동을 펼치고 있다.
line 3 : 가충순 의원은 5분발언, 행정사무감사, 시정질문을 통해 자동차 연비테스트 연구시설 유치, 천수만 염해피해 재발 방지, 서산시 대표 농산물 육성 등 지역의 크고 작은 문제를 개선하기 위해 노력하고 있다.
line 4 : 이수의 의원은 지난 행정사무감사에서 대산공단 기업 임원을 참고인으로 출석시켜 지역인재채용 및 관내업체·자재 활용을 확대할 것을 제안하며 기존 행정사무감사의 틀을 깨는 등 다양한 의정활동을 펼쳐나가고 있다.
line 5 : 가충순 의원은 "시의원이라면 마땅히 해야할 일을 한 것 뿐인데 상까지 주시니 몸 둘 바를 모르겠다"며 "항상 초심을 잊지 않고 지역 발전을 위해 최선을 다하겠다"고 소감을 밝혔다.
line 6 : 이수의 의원은 "믿고 뽑아주신 주민들을 위해 당연히 해야할 일을 했을 뿐인데 상까지 받게 돼 영광"이라며 "시민들이 자부심을 느낄 수 있는 지역사회를 만들어 나가기 위해 앞으로도 최선을 다 하겠다"고 말했다.


In [ ]:
def articles_to_str(articles_list):
  return ' '.join(articles_list)

In [ ]:
train_df['article_to_str']=train_df['article_original'].apply(articles_to_str)

In [ ]:
train_df.head()

,media,id,article_original,abstractive,extractive,article_to_str
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]",지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성...
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]",서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 우...
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]",지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대장...
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]",서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 2...
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]",미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민단...


In [ ]:
print('Length of original dataset:', len(train_df))
train_small = train_df[:1000]
print('Length of truncated dataset:', len(train_small))

Length of original dataset: 260697
Length of truncated dataset: 1000


In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

1. t5 모델 base로 했는데도 큼

In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[labels[:, :] == tokenizer.pad_token_id] = -100
        
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
train_small.rename(columns = {'abstractive' : 'ctext',
                              'article_to_str' : 'text'}, inplace = True)
train_small.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,media,id,article_original,ctext,extractive,text
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]",지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성...
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]",서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 우...
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]",지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대장...
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]",서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 2...
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]",미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민단...


In [ ]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="kubig_t5_summarization")

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 5        # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 2 
    config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 150 

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    model_name = 'KETI-AIR/ke-t5-base'
    # tokenzier for encoding the text
    # tokenizer = T5Tokenizer.from_pretrained("t5-base")
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
    df = train_small[['ctext','text']]
    df.ctext = 'summarize: ' + df.ctext
    print(df.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset:')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe:')
    for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('./t5_predictions_1.csv')
        print('Output Files generated for review.')

if __name__ == '__main__':
    main()